In [11]:
import astropy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from time import time
from astropy.io import fits
from time import time
from time import time
from collections import Counter,defaultdict
import json
import pickle
%matplotlib inline

In [12]:
# path to database file
db_path = "/Users/Alexander/NASA/DataBase/db_features.txt"

###Get Extracted Features from File

In [19]:
def get_features(path):
    file_names = []
    feat_extract_list = []
    for result in os.listdir(path):
        if "ds_store" not in result.lower():
            file_names.append(result)
            print path + result
            feat_extract_list.append(pd.read_pickle(path + result).values)
    return feat_extract_list, file_names

In [20]:
path_results = "/Users/Alexander/NASA/feature_extraction_results_test/"
feats, feat_file_names = get_features(path_results)

/Users/Alexander/NASA/feature_extraction_results_test/fd_M_96m_01d_3925_0000.pk
/Users/Alexander/NASA/feature_extraction_results_test/fd_M_96m_01d_3925_0001.pk
/Users/Alexander/NASA/feature_extraction_results_test/fd_M_96m_01d_3925_0002.pk


#### This is the object that the Extract Features function will output

In [21]:
# This is the object that the Extract Features function will output 
feats[0]

array([['fd_M_96m_01d.3925.0000.fits',
        {1: {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}, 2: {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}, 3: {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}, 4: {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}, 'image_type': 'MDI', 'image_time': '00:03:03', 'car_long': 297.3335268426, 'image_date': '2003 10 01'}]], dtype=object)

###Format Results and Save to DB file

In [22]:
fresh_results = feats[0]

In [23]:
fresh_results

array([['fd_M_96m_01d.3925.0000.fits',
        {1: {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}, 2: {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}, 3: {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}, 4: {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}, 'image_type': 'MDI', 'image_time': '00:03:03', 'car_long': 297.3335268426, 'image_date': '2003 10 01'}]], dtype=object)

In [24]:
for key, value in fresh_results[0][1].iteritems():
    print key, value

1 {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}
2 {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}
3 {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}
4 {'y_pos_ave': 528.77, 'x_pos_ave': 990.17, 'pos_net_flux': 1574165.5291137344, 'neg_net_flux': -478205.4023385141, 'y_neg_ave': 524.55, 'x_neg_ave': 961.2}
image_type MDI
image_time 00:03:03
car_long 297.333526843
image_date 2003 10 01


In [ ]:
# Notes from talk with David - Jan 22, 2016
# Saving results to text file
# Image Name should HMIYYYY.txt (i.e. discripter date . txt )
# header: |YYYY MM DD| |hh:mm:ss| |Active Region #| |Flux| |Longitude| |Latitude|
#                                  xxxxxx[letter]     +/-     xxx.xx     xxx.xx
# unidentifed active regions, label them starting at 100,000 
# will have to map pixels to longitude and latitude 
# can provide lead digit to indicate that we have observed an active region that NOAA has not
# we want the carinton longitude 

###Convert Cartesian Coordinates to Longitude & Latitude (degrees)
[a, a, a, ....a]

    Longitude = clon1 + (xbar - 511.5) * 180/1024 MDI
                        (xbar - 2047.5)*180/4096  HMI

    Longitude = (Longitude + 360) mod 360

    Latitude = artsin[ (ybar - 511.5)/512]   MDI
    Latitude = artsin[ (ybar - 2047.5)/2048] HMI
    Latitude = Latitude * 180/pi (conversion from radius to degrees)


    For Database 2.0
    0. disentanlge complex clusters of active regions
        consider time series
            changing centroids in time can indicate emerange of new active regions/new sunspots
        consider clustering 
    
    
  
1. Perform conversion (Lat, Long)
2. Create Database 1.0